## Cloning github repositry that contains Tacotron 2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext autoreload

In [5]:
!git clone https://github.com/Fadi-S/tts-arabic-pytorch

Cloning into 'tts-arabic-pytorch'...
remote: Enumerating objects: 404, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 404 (delta 56), reused 57 (delta 43), pack-reused 320
Receiving objects: 100% (404/404), 5.00 MiB | 7.11 MiB/s, done.
Resolving deltas: 100% (219/219), done.


## 🔴 Only run after updating tts repo

In [ ]:
!git pull origin master
%autoreload 2

## Downloading Pretrained model

### 🔴 Not necessary if you wont try to run the last cell, we have our own states now...

In [ ]:
!gdown 1FD2J-xUk48JPF9TeS8ZKHzDC_ZNBfLd8

Downloading...
From (original): https://drive.google.com/uc?id=1FD2J-xUk48JPF9TeS8ZKHzDC_ZNBfLd8
From (redirected): https://drive.google.com/uc?id=1FD2J-xUk48JPF9TeS8ZKHzDC_ZNBfLd8&confirm=t&uuid=4dba95c8-a80d-4e6c-bdb7-df8dbce1c29f
To: /content/ar-tts-models.zip
100% 633M/633M [00:11<00:00, 55.1MB/s]


In [ ]:
!unzip -o ar-tts-models.zip -d tts-arabic-pytorch/pretrained/

Archive:  ar-tts-models.zip
   creating: tts-arabic-pytorch/pretrained/diacritizers/
  inflating: tts-arabic-pytorch/pretrained/diacritizers/shakkala_second_model6.pth  
  inflating: tts-arabic-pytorch/pretrained/diacritizers/shakkelha_rnn_3_big_20.pth  
  inflating: tts-arabic-pytorch/pretrained/fastpitch_ar_adv.pth  
  inflating: tts-arabic-pytorch/pretrained/fastpitch_ar_mse.pth  
  inflating: tts-arabic-pytorch/pretrained/hifigan-asc-v1/config.json  
  inflating: tts-arabic-pytorch/pretrained/hifigan-asc-v1/hifigan-asc.pth  
  inflating: tts-arabic-pytorch/pretrained/tacotron2_ar_adv.pth  
  inflating: tts-arabic-pytorch/pretrained/tacotron2_ar_mse.pth  


## Downloading and pre-processing dataset


In [ ]:
!unzip -o "/content/drive/MyDrive/Handassa/Term 8/Deep Learning/mo5ber/cut_clips9.zip" -d /content

In [6]:
import os
os.chdir('/content/tts-arabic-pytorch')

In [7]:
import pandas as pd

dataset_path = '/content/cut_clips9'

# Load the index.csv file into a DataFrame
data = pd.read_csv('/content/cut_clips9/segments.csv', names=['audio_file', 'text'], dtype=str)

audio_files = data['audio_file']
texts = data['text']

# Display the contents of index.csv
print(data.head())

  audio_file                                    text
0      00001          هل التطبيق الصيني المعروف باسم
1      00002                                  تيكتوك
2      00003                هو أخطر تطبيق في العالم 
3      00004  السؤال دا مهم جداً ولازم كلنا نفكر فيه
4      00005        وهو في الحقيقة شاغل بال ناس كتير


In [8]:
from sklearn.model_selection import train_test_split


# Split the data into training and validation sets
train_audio_files, val_audio_files, train_texts, val_texts = train_test_split(audio_files, texts, test_size=0.2, random_state=42)

train_data = pd.DataFrame({'filename': train_audio_files, 'text': train_texts})
val_data = pd.DataFrame({'filename': val_audio_files, 'text': val_texts})

In [9]:
train_data = pd.DataFrame(train_data, columns=['filename', 'text'])
val_data = pd.DataFrame(val_data, columns=['filename', 'text'])

# Display the data frame
train_data.head(10)

,filename,text
601,00602,فيما يخص تشييد الأبنية المقاومة للزلازل
9397,09398,ومستثمرين بيستثمروا في أدوات الدين
13447,13448,ومش عاوزين يدفعوا ليه
25666,25667,شيء مذهل ما بيحصلش حالياً
27958,27959,أو درع السيليكون
29475,29476,شركة سيارات في العالم بعد ما مبيعاتها
14331,14332,الأمريكيين كانوا مستعدين نفسياً لأنهم
7582,07583,بعد ما الحرب انتهت جزء كبير من الفلاحين
22974,22975,المهم
7722,07723,والتعليم


In [10]:
import text

def preprocess(txt):
    txt = txt.replace(".", "")
    txt = txt.replace("!", "")
    txt = txt.replace(",", "")
    t_phon = text.arabic_to_buckwalter(txt)
    t_phon = text.buckwalter_to_phonemes(t_phon)
    return t_phon

In [11]:
train_phonemes,val_phonemes = [], []
train_path,val_path = [], []


for _, row in train_data.iterrows():
    filename = "ARA NORM " + str(row['filename']) + '.wav'
    train_path.append(filename)
    train_phonemes.append(preprocess(row['text']))

for _,row in val_data.iterrows():
    row['filename'] = "ARA NORM " + str(row['filename']) + '.wav'
    val_path.append(filename)
    val_phonemes.append(preprocess(row['text']))

In [12]:
train_data = train_data.assign(Phonemes=train_phonemes)
val_data = val_data.assign(Phonemes=val_phonemes)

train_data = train_data.assign(filename=train_path)
val_data = val_data.assign(filename=val_path)

In [13]:
train_data = train_data.drop('text', axis=1)
val_data = val_data.drop('text', axis=1)

In [14]:
train_data.head()

,filename,Phonemes
601,ARA NORM 00602.wav,+ f ii0 m aa + ii0 x S + t $ y ii0 d + l < b ...
9397,ARA NORM 09398.wav,uu0 m s t ^ m r ii0 n + b ii0 s t ^ m r w aa +...
13447,ARA NORM 13448.wav,uu0 m $ + E aa uu0 z ii0 n + ii0 d f E w aa + ...
25666,ARA NORM 25667.wav,$ ii0 < + m * h l + m aa + b ii0 H S l $ + H a...
27958,ARA NORM 27959.wav,< uu0 + d r E + l s ii0 l ii0 k uu0 n


In [15]:
import csv

train_data.to_csv('/content/ready_train.txt', sep=' ', header=None, index=None, quoting=csv.QUOTE_NONNUMERIC)
val_data.to_csv('/content/ready_val.txt', sep=' ', header=None, index=None, quoting=csv.QUOTE_NONNUMERIC)

In [16]:
#reading path for wav dataset
train_path = "/content/cut_clips9"
val_path = "/content/cut_clips9"

In [17]:
from utils.data import ArabDataset


train_dataset = ArabDataset('/content/ready_train.txt', train_path)
test_dataset = ArabDataset('/content/ready_val.txt', val_path)

Number of lines: 26172


100%|██████████| 26172/26172 [00:00<00:00, 28625.71it/s]


Number of mel phonemes: 26172
Number of lines: 6544


100%|██████████| 6544/6544 [00:00<00:00, 34863.03it/s]


Number of mel phonemes: 6544


In [27]:
def compute_accuracy(gate_out, gate_padded):
    # Apply sigmoid function to gate_out to get probabilities
    gate_prob = torch.sigmoid(gate_out)
    # Round the probabilities to get binary predictions (0 or 1)
    gate_pred = torch.round(gate_prob)
    # Compute accuracy by comparing predictions with targets
    correct = (gate_pred == gate_padded).sum().item()
    total = gate_padded.numel()
    accuracy = correct / total
    return accuracy

# FineTuning model

## Training and Validation loop

In [35]:
import torch
import torch.nn.functional as F
import random
from utils.training import batch_to_device, save_states
from tqdm import tqdm

@torch.inference_mode()
def validate(model, test_loader, device, n_iter):
    loss_sum = 0
    n_test_sum = 0

    model.eval()

    for batch in test_loader:
        text_padded, input_lengths, mel_padded, gate_padded, output_lengths = batch_to_device(batch, device)

        y_pred = model(text_padded, input_lengths, mel_padded, output_lengths, torch.zeros_like(output_lengths))

        mel_out, mel_out_postnet, gate_pred, alignments = y_pred

        mel_loss = F.mse_loss(mel_out, mel_padded) + F.mse_loss(mel_out_postnet, mel_padded)
        gate_loss = F.binary_cross_entropy_with_logits(gate_pred, gate_padded)
        loss = mel_loss + gate_loss

        loss_sum += mel_padded.size(0) * loss.item()
        n_test_sum += mel_padded.size(0)

    val_loss = loss_sum / n_test_sum

    idx = random.randint(0, mel_padded.size(0) - 1)
    mel_infer, *_ = model.infer(text_padded[idx:idx+1], input_lengths[idx:idx+1]*0, input_lengths[idx:idx+1])

    model.train()

    return val_loss

def training_loop(model,
                  optimizer,
                  train_loader,
                  test_loader,
                  device,
                  config,
                  n_epoch,
                  n_iter):

    model.train()
    net_config = {}
    for epoch in range(n_epoch, config.epochs):

        with tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs}", unit="batch") as t:
            for batch in t:

                text_padded, input_lengths, mel_padded, gate_padded, output_lengths = batch_to_device(batch, device)

                y_pred = model(text_padded, input_lengths, mel_padded, output_lengths, torch.zeros_like(output_lengths))
                mel_out, mel_out_postnet, gate_out, _ = y_pred

                optimizer.zero_grad()

                # LOSS
                mel_loss = F.mse_loss(mel_out, mel_padded) + F.mse_loss(mel_out_postnet, mel_padded)
                gate_loss = F.binary_cross_entropy_with_logits(gate_out, gate_padded)
                loss = mel_loss + gate_loss

                loss.backward()

                grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_clip_thresh)
                optimizer.step()

                accuracy = compute_accuracy(gate_out, gate_padded)
                t.set_postfix(loss=loss.item(), grad_norm=grad_norm.item(), accuracy=accuracy)

                # LOGGING and saving weights
                if n_iter % config.n_save_states_iter == 0:
                    save_states(f'states.pth', model, optimizer, n_iter, epoch, net_config, config)

                if n_iter % config.n_save_backup_iter == 0 and n_iter > 0:
                    save_states(f'states_{n_iter}.pth', model, optimizer, n_iter, epoch, net_config, config)

                n_iter += 1

        # VALIDATE
        val_loss = validate(model, test_loader, device, n_iter)
        print(f"Validation loss: {val_loss}")

In [36]:
class config(object):

    def __init__ (self, batch=8, epochs = 500, grad_clip_thresh = 1.0, learning_rate = 1.0e-5, weight_decay = 1.0e-8, max_step = 3000):
        self.epochs = epochs
        self.grad_clip_thresh = grad_clip_thresh
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.max_step = max_step
        self.batch = batch
        self.n_save_states_iter = 100
        self.n_save_backup_iter = 1000
        self.checkpoint_dir = "/content/drive/MyDrive/Handassa/Term 8/Deep Learning/Checkpoint"
        self.pretrained_dir = "/content/drive/MyDrive/Handassa/Term 8/Deep Learning/Checkpoint/states.pth"


In [37]:
config = config(batch=64, learning_rate=1.0e-3)

In [45]:
from torch.utils.data import DataLoader
from utils.data import text_mel_collate_fn

# dataloaders
train_loader = DataLoader(train_dataset,
                              batch_size=config.batch,
                              collate_fn=text_mel_collate_fn,
                              shuffle=True, drop_last=True,
                              sampler=None)

test_loader = DataLoader(test_dataset,
                             batch_size=config.batch, drop_last=False,
                             shuffle=False, collate_fn=text_mel_collate_fn)

## Model architecture, we used Tacotron2 as our base model

In [ ]:
from models.tacotron2.tacotron2_ms import Tacotron2MS

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# construct model
model = Tacotron2MS(n_symbol=40)
model = model.to(device)
model.decoder.decoder_max_step = config.max_step

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

# resume from existing checkpoint
n_epoch, n_iter = 0, 0


state_dicts = torch.load(config.pretrained_dir, map_location=device)
model.load_state_dict(state_dicts['model'])
if 'optim' in state_dicts:
      optimizer.load_state_dict(state_dicts['optim'])
if 'epoch' in state_dicts:
      n_epoch = state_dicts['epoch']
if 'iter' in state_dicts:
      n_iter = state_dicts['iter']


# start training
training_loop(model, optimizer, train_loader, test_loader, device, config, n_epoch, n_iter)

Epoch 11/500:  77%|███████▋  | 316/408 [15:05<03:55,  2.56s/batch, accuracy=0.995, grad_norm=0.879, loss=0.665]

In [ ]:
import matplotlib.pyplot as plt
import IPython
from models.tacotron2 import Tacotron2Wave

model = Tacotron2Wave("/content/drive/MyDrive/Handassa/Term 8/Deep Learning/Checkpoint/states.pth")
model = model.cuda()

wave, mel_spec = model.tts("ازيك عامل ايه يا باشا" ,return_mel=True, denoise=0.005)

print("Audio output (Tacotron2)")
IPython.display.Audio(data=0.5*wave, rate=44100, normalize=False)